# Import

In [1]:
import json
import os
import shutil
from tqdm import tqdm
from glob import glob

# convert func

In [6]:
def obj_class(data):
    if(data == "플라스틱류"):
        return 0
    elif(data == "캔류"):
        return 1
    elif(data == "종이류"):
        return 2
    elif(data == "유리병류"):
        return 3
    elif(data == "페트병류"):
        return 4

In [3]:
def convert(json_datas, yolo_txt_path, yolo_img_path):
    for json_data in tqdm(json_datas):
        filename = os.path.basename(json_data).split('.')[0]
        yolo = open(f'{yolo_txt_path}/{filename}.txt','w')
        
        with open(json_data, 'r', encoding="utf-8") as f:
            json_data_parse = json.load(f)
        
        width, height = list(map(int,json_data_parse["RESOLUTION"].split("*")))
        
        check = False
        
        for obj in json_data_parse["Bounding"]:
            if(obj["Drawing"]=="BOX"):
                met = obj_class(obj["CLASS"])
                x1 = int(obj["x1"])
                y1 = int(obj["y1"])
                x2 = int(obj["x2"])
                y2 = int(obj["y2"])

                dw = 1./width
                dh = 1./height
                x = (x1 + x2)/2.0
                y = (y1 + y2)/2.0
                w = x2 - x1
                h = y2 - y1
                x = x*dw
                w = w*dw
                y = y*dh
                h = h*dh

                yolo.write(f'{met} {x:.6f} {y:.6f} {w:.6f} {h:.6f}\n')
                check = True
        yolo.close()
        
        try:
            if(check):
                shutil.copyfile(f'{os.path.dirname(json_data)}/{filename}.jpg'.replace("json","img"),f'{yolo_img_path}/{filename}.jpg')
            else:
                os.remove(f'{yolo_txt_path}/{filename}.txt')
        except:
            os.remove(f'{yolo_txt_path}/{filename}.txt')
            

# File Load

In [4]:
train_img_path = "/home/jovyan/work/Training/img"
train_json_path = "/home/jovyan/work/Training/json"
train_yolo_img_path = "/home/jovyan/work/Training/yolo/img"
train_yolo_txt_path = "/home/jovyan/work/Training/yolo/txt"
train_img = glob(train_img_path+"/*/*.jpg")
train_json = glob(train_json_path+"/*/*.Json")

test_img_path = "/home/jovyan/work/Validation/img"
test_json_path = "/home/jovyan/work/Validation/json"
test_yolo_img_path = "/home/jovyan/work/Validation/yolo/img"
test_yolo_txt_path = "/home/jovyan/work/Validation/yolo/txt"
test_img = glob(test_img_path+"/*/*.jpg")
test_json = glob(test_json_path+"/*/*.Json")

print('-------train dataset------')
print('img : '+str(len(train_img)))
print('json : '+str(len(train_json)))

print('-------test dataset------')
print('img : '+str(len(test_img)))
print('json : '+str(len(test_json)))

-------train dataset------
img : 167543
json : 203122
-------test dataset------
img : 25426
json : 25426


# Convert

In [7]:
convert(train_json, train_yolo_txt_path, train_yolo_img_path)
convert(test_json, test_yolo_txt_path, test_yolo_img_path)

100%|████████████████████████████████████████████████████████████| 25426/25426 [01:13<00:00, 343.69it/s]
